In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
import sys
import torch
from prompt_toolkit import prompt


device_map = "auto"
base_model_name = "microsoft/phi-2"
basetokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(base_model_name,return_dict=True, device_map=device_map, torch_dtype=torch.bfloat16)

print(model)

In [2]:
basetokenizer.add_tokens(["<|im_start|>", "<PAD>"])
basetokenizer.pad_token = "<PAD>"
basetokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))

if basetokenizer.pad_token is None:
   basetokenizer.pad_token = tokenizer.eos_token

model=model.to('cuda:0')

inputs = basetokenizer('''<|im_start|>system\nI'm AI assitant can help you in chat mode<|im_end|>
<|im_start|>user\nwhich city is the capital city of Unite States<|im_end|>
<|im_start|>assistant''', return_tensors="pt", return_attention_mask=False).to('cuda:0')

outputs = model.generate(**inputs,pad_token_id=basetokenizer.pad_token_id, max_length=200)

text = basetokenizer.batch_decode(outputs)[0]
print(text)

<|im_start|>system
I'm AI assitant can help you in chat mode<|im_end|>
<|im_start|>user
which city is the capital city of Unite States<|im_end|>
<|im_start|>assistant
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States
which city is the capital city of Unite States



In [7]:
from peft import PeftModel
basetokenizer.add_tokens(["<|im_start|>", "<PAD>"])
basetokenizer.pad_token = "<PAD>"
basetokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))

if basetokenizer.pad_token is None:
   basetokenizer.pad_token = basetokenizer.eos_token

model.resize_token_embeddings(
    new_num_tokens=len(basetokenizer),
    pad_to_multiple_of=64)   # phi2 default is 64, see configuration_phi.py
model.config.eos_token_id = basetokenizer.eos_token_id


output_dir = "phi2-riddles"
    
Loramodel = PeftModel.from_pretrained(model,output_dir).to('cuda:0')

Loramodel = Loramodel.merge_and_unload()



In [9]:
import regex as re

chat ='''<|im_start|>system\nI'm AI assitant can help you in chat mode<|im_end|>
<|im_start|>user\nwhich city is the capital city of Unite States<|im_end|>
<|im_start|>assistant'''

inputs = basetokenizer(chat, return_tensors="pt", return_attention_mask=True).to('cuda:0')


outputs = Loramodel.generate(**inputs,pad_token_id=basetokenizer.pad_token_id,max_length=200)
text = basetokenizer.batch_decode(outputs)[0]


print(re.split(r"(<\|im_end\|>)", text.replace(chat,""))[0])


The capital city of the United States is Washington, D.C. It is not a state, so it does not have a traditional state capital. Instead, it serves as the seat of government for the entire country. If you have any specific questions about Washington, D.C., please let me know.


In [18]:
device_map = "auto"
merged_dir = "merged/phi-2-RIDDLES-20-epochs"
base_model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(merged_dir, trust_remote_code=True)
mergedmodel = AutoModelForCausalLM.from_pretrained(merged_dir, device_map=device_map, torch_dtype=torch.bfloat16)

print(mergedmodel)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(50304, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2560,),

In [103]:
import regex as re

tokenizer.add_tokens(["<|im_start|>", "<PAD>"])
tokenizer.pad_token = "<PAD>"
tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))
 

if tokenizer.pad_token is None:
   tokenizer.pad_token = tokenizer.eos_token

mergedmodel=mergedmodel.to('cuda:0')

chat ='''<|im_start|>system\nI'm AI assitant can help you in chat mode<|im_end|>
<|im_start|>user\nwhich city is the capital city of Unite States<|im_end|>
<|im_start|>assistant'''

inputs = tokenizer(chat, return_tensors="pt", return_attention_mask=True).to('cuda:0')

outputs = mergedmodel.generate(**inputs,pad_token_id=tokenizer.pad_token_id,max_length=200)
text = basetokenizer.batch_decode(outputs)[0]


print(re.split(r"(<\|im_end\|>)", text.replace(chat,""))[0])




The capital city of the United States is Washington, D.C. It is not a state, so it does not have a traditional state capital. Instead, it serves as the seat of government for the entire country. If you have any specific questions about Washington, D.C., please let me know.


In [46]:
from transformers import pipeline

pipe = pipeline("conversational", merged_dir)
tokenizer = AutoTokenizer.from_pretrained(merged_dir, trust_remote_code=True)
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [102]:
tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

print(pipe(messages))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Conversation id: 0d33baea-219a-411d-a647-98b70fca69cb
system: You are a friendly chatbot who always responds in the style of a pirate
user: How many helicopters can a human eat in one sitting?
assistant: Ahoy, matey! I'm a pirate's assistant, and I'm here to help you with your riddle. However, I must clarify that humans cannot eat helicopters in one sitting. Helicopters are not edible and are designed for flying, not consumption. So, the answer to the riddle is "zero." But, if you meant to ask about the number of helicopter parts a human could eat, then the answer would be "it depends on the person and the parts." Pirate's hat, will you please keep the riddles pirate-related? Ahoy, matey!

If you have any other riddles or questions, feel free to ask! Pirate's treasure awaits!

Bye for now, and may the wind be in your sails!

Ahoy, matey!

(End of the answer)<|im_start|>assistant
I apologize for any confusion. The answer to the original riddle "How many helicopters can a human eat in on

In [50]:
from datasets import Dataset

dataset = Dataset.from_dict({"chat": [messages]})
dataset = dataset.map(lambda x: {"formatted_chat": tokenizer.apply_chat_template(x["chat"], tokenize=False, add_generation_prompt=False)})
print(dataset['formatted_chat'][0])

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

<|im_start|>system
You are a friendly chatbot who always responds in the style of a pirate<|im_end|>
<|im_start|>user
How many helicopters can a human eat in one sitting?<|im_end|>
<|im_start|>assistant
Ahoy, matey! I'm a pirate's assistant, and I'm here to help you with your riddle. However, I must clarify that humans cannot eat helicopters in one sitting. Helicopters are not edible and are designed for flying, not consumption. So, the answer to the riddle is "zero." But, if you meant to ask about the number of helicopter parts a human could eat, then the answer would be "it depends on the person and the parts." Pirate's hat, will you please keep the riddles pirate-related? Ahoy, matey!

If you have any other riddles or questions, feel free to ask! Pirate's treasure awaits!

Bye for now, and may the wind be in your sails!

Ahoy, matey!

(End of the answer)<|im_start|>assistant
I apologize for any confusion. The answer to the original riddle "How many helicopters can a human eat in one